In [77]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, CuDNNLSTM, LSTM, SpatialDropout1D, GlobalMaxPool1D, Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
import re
import seaborn as sns
import matplotlib.pyplot as plt
import string   

In [46]:
data = pd.read_csv('./fake-news/train.csv', delimiter=",", encoding='utf-8')

In [47]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [48]:
data['text'] = data['author'] + " " + data['title'] + " " +  data['text']

In [49]:
data = data.drop(['id','title','author'],1)

<ipython-input-49-7487443a8a60>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop(['id','title','author'],1)


In [50]:
import re
from typing import List

import spacy
from spacy.tokens import Doc
from tqdm import tqdm


class SpacyPreprocessor:
    def __init__(
        self,
        spacy_model=None,
        remove_numbers=False,
        remove_special=True,
        pos_to_remove=None,
        remove_stopwords=False,
        lemmatize=False,
    ):
        """
        Preprocesses text using spaCy
        :param remove_numbers: Whether to remove numbers from text
        :param remove_stopwords: Whether to remove stopwords from text
        :param remove_special: Whether to remove special characters (including numbers)
        :param pos_to_remove: list of PoS tags to remove
        :param lemmatize:  Whether to apply lemmatization
        """

        self._remove_numbers = remove_numbers
        self._pos_to_remove = pos_to_remove
        self._remove_stopwords = remove_stopwords
        self._remove_special = remove_special
        self._lemmatize = lemmatize

        if not spacy_model:
            self.model = spacy.load("en_core_web_sm")
        else:
            self.model = spacy_model

    @staticmethod
    def download_spacy_model(model="en_core_web_sm"):
        print(f"Downloading spaCy model {model}")
        spacy.cli.download(model)
        print(f"Finished downloading model")

    @staticmethod
    def load_model(model="en_core_web_sm"):
        return spacy.load(model, disable=["ner", "parser"])

    def tokenize(self, text) -> List[str]:
        """
        Tokenize text using a spaCy pipeline
        :param text: Text to tokenize
        :return: list of str
        """
        doc = self.model(text)
        return [token.text for token in doc]

    def preprocess_text(self, text) -> str:
        """
        Runs a spaCy pipeline and removes unwanted parts from text
        :param text: text string to clean
        :return: str, clean text
        """
        doc = self.model(text)
        return self.__clean(doc)

    def preprocess_text_list(self, texts=List[str]) -> List[str]:
        """
        Runs a spaCy pipeline and removes unwantes parts from a list of text.
        Leverages spaCy's `pipe` for faster batch processing.
        :param texts: List of texts to clean
        :return: List of clean texts
        """
        clean_texts = []
        for doc in tqdm(self.model.pipe(texts)):
            clean_texts.append(self.__clean(doc))

        return clean_texts

    def __clean(self, doc: Doc) -> str:

        tokens = []
        # POS Tags removal
        if self._pos_to_remove:
            for token in doc:
                if token.pos_ not in self._pos_to_remove:
                    tokens.append(token)
        else:
            tokens = doc

        # Remove Numbers
        if self._remove_numbers:
            tokens = [
                token for token in tokens if not (token.like_num or token.is_currency)
            ]

        # Remove Stopwords
        if self._remove_stopwords:
            tokens = [token for token in tokens if not token.is_stop]
        # remove unwanted tokens
        tokens = [
            token
            for token in tokens
            if not (
                token.is_punct or token.is_space or token.is_quote or token.is_bracket
            )
        ]

        # Remove empty tokens
        tokens = [token for token in tokens if token.text.strip() != ""]

        # Lemmatize
        if self._lemmatize:
            text = " ".join([token.lemma_ for token in tokens])
        else:
            text = " ".join([token.text for token in tokens])

        if self._remove_special:
            # Remove non alphabetic characters
            text = re.sub(r"[^a-zA-Z\']", " ", text)
        # remove non-Unicode characters
        text = re.sub(r"[^\x00-\x7F]+", "", text)

        text = text.lower()

        return text

In [57]:
preprocessor = SpacyPreprocessor(remove_numbers=True,
                                 remove_special=True,
#                                 pos_to_remove=True,
                                 remove_stopwords=True,
                                 lemmatize=True)
data['text'] = data['text'].apply(lambda x: preprocessor.preprocess_text(str(x)))

In [29]:
data.head()

,text,label
0,darrell lucus house dem aide we didn’t even se...,1
1,daniel j flynn flynn hillary clinton big woman...,0
2,consortiumnewscom why the truth might get you ...,1
3,jessica purkiss civilians killed in single us...,1
4,howard portnoy iranian woman jailed for fictio...,1


In [59]:
words = set(" ".join(data['text'].tolist()).split(' '))

In [60]:
words

{'',
 'substitute',
 'maxwel',
 'lutein',
 'falla',
 'cratchette',
 'wenezuel',
 'polaroid',
 'whyatt',
 'shyer',
 'votetoday',
 'creativo',
 'serlo',
 'iaf',
 'kaffiyeh',
 'overtreatment',
 'nonstick',
 'mpft',
 'fram',
 'hajaja',
 'multicity',
 'frontal',
 'purine',
 'exsactly',
 'portraitist',
 'lomonte',
 'venusino',
 'insolvent',
 'hockenbury',
 'madiran',
 'conducir',
 'uncontested',
 'amerisleep',
 'ndeme',
 'gallagher',
 'iaayh',
 'forfeits',
 'niebla',
 'palumbo',
 'talker',
 'patronal',
 'rottentomatoes',
 'hamlyn',
 'thirdworldize',
 'masshole',
 'enministeriums',
 'offrir',
 'obliterate',
 'kissinger',
 'injun',
 'unhealthy',
 'shishapangma',
 'bijan',
 'jamshoro',
 'precipitant',
 'johannsson',
 'berankis',
 'euler',
 'wissen',
 'longo',
 'decolonization',
 'poaching',
 'aliquippa',
 'bonquita',
 'yousefi',
 'llevado',
 'greenwave',
 'nullification',
 'inventer',
 'swinging',
 'politicizing',
 'lauric',
 'ld',
 'pretentiously',
 'terawatt',
 'emidio',
 'anbd',
 'events',
 

In [64]:
data['wordCount'] = data['text'].str.split().str.len() # Populating work count per news

In [68]:
max_features = len(words) # Number of distinct words in news 
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
#X_test = tokenizer.texts_to_sequences(testData['comment_text'].values)
X = pad_sequences(X)
#X_test = pad_sequences(X_test)

array([[    0,     0,     0, ..., 13955, 25504,  1248],
       [    0,     0,     0, ...,  1682,   945,  7230],
       [    0,     0,     0, ...,   256,  1508,  6413],
       ...,
       [    0,     0,     0, ...,  3416,  2865,   611],
       [    0,     0,     0, ...,  2256,  1388,  2804],
       [    0,     0,     0, ..., 14365,  1818,  1714]], dtype=int32)

In [90]:
maxlen = max(data['wordCount']) # max number of words per news   
embed_dim = 128
model = Sequential([
    Embedding(input_dim=max_features, output_dim=embed_dim, 
                              input_length=maxlen),
    LSTM(60, return_sequences=True),
    GlobalMaxPool1D(),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dropout(0.1),
    Dense(1, activation='sigmoid')])    # only 2 value in each labels

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# using binary crossentropy because each labels or the features only have 2 value, 0 or 1

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 10578, 128)        16590208  
_________________________________________________________________
lstm_4 (LSTM)                (None, 10578, 60)         45360     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 60)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3050      
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [84]:
Y = data[['label']]

In [ ]:
batch_size = 128
model.fit(X, Y, epochs = 10, batch_size=batch_size, verbose = 1)

Epoch 1/10
 85/163 [==============>...............] - ETA: 14:26 - loss: 0.5585 - accuracy: 0.7425